# Summary

### In this notebook we look to evaluate if the utility (usefullness) of embeddings is preserved, improved, or degraded by performing concept categorization and analogy analysis. Techniques for these analysis are adapted from methods implemented in https://github.com/uvavision/Double-Hard-Debias

In [1]:
import gensim.downloader as api
import json
import os
import numpy as np

from util import load_legacy_w2v, pruneWordVecs, convert_legacy_to_keyvec
from neighborAnalysis import get_most_biased, cluster
from loader import load_def_sets, load_analogy_templates, load_eval_terms
from biasOps import identify_bias_subspace, project_onto_subspace, neutralize_and_equalize, calculate_main_pca_components,neutralize_and_equalize_with_frequency_removal
from evalBias import generateAnalogies, multiclass_evaluation
from sklearn.decomposition import PCA
from concept import evaluate_cate, create_bunches
from analogy import evaluate_analogies

In [2]:
import gensim
gensim.__version__  # I use '3.6.0'

'4.1.2'

# 1) Process Embeddings

## Obtain Pre-trained Word2Vec Embedding

In [3]:
# Pretrained embeddings from the l2 reddit corpus
# https://github.com/TManzini/DebiasMulticlassWordEmbedding
reddit_path = os.path.join('data', 'data_vocab_race_pre_trained.w2v')

## Perform Hard Debias

Two steps:
1. Identify the bias subspace
2. remove this component from the set of embeddings

In [4]:
# Load unbiased embeddings
reddit_embeddings, embedding_dim = load_legacy_w2v(reddit_path, 50)

# take out words that do not only contain letters
word_vectors = pruneWordVecs(reddit_embeddings)
print("Remaining words after prune:", len(word_vectors))

# Get defining sets
def_sets = load_def_sets('data/vocab/race_attributes_optm.json')


# Calculate the bias subspace
subspace = identify_bias_subspace(word_vectors, def_sets, 2, embedding_dim)

# Get neutral words
roles = load_analogy_templates('data/vocab/race_attributes_optm.json','role')

neutral_words = []
for value in roles.values():
    neutral_words.extend(value)

# use the defining set as the equality set
eq_sets = [set for _, set in def_sets.items()]    
new_vocab = neutralize_and_equalize(word_vectors, neutral_words, eq_sets, subspace, embedding_dim)    

Remaining words after prune: 44895


## Perform Double-Hard Debias

In [5]:
subspace = identify_bias_subspace(word_vectors, def_sets, 2, embedding_dim)

# get all positive and negative words words within the subspace (moving towards or away from some race)
positive_words, negative_words = get_most_biased(word_vectors, subspace[0], 1000)

# get all biased words (minus those in the defining/equality set)
positive_words = [word for word, sim in positive_words]
negative_words = [word for word, sim in negative_words]

biased_words = set(positive_words + negative_words) - set(np.array(eq_sets).flatten())

# 1 for more positive biased words, 0 for more negative biased words
y_true = [ 1 if word in positive_words else 0 for word in biased_words]

In [6]:
# calculate decentralized embedding pca
main_pca = calculate_main_pca_components(convert_legacy_to_keyvec(word_vectors)).components_

# run debias for each principal component and capture best precision
precisions = []
debiases = []

for pc in main_pca:
    debiased_frequency = neutralize_and_equalize_with_frequency_removal(word_vectors, biased_words, eq_sets, subspace, embedding_dim, pc)
    x_dhd = [debiased_frequency[word] for word in biased_words]    
    precisions.append(cluster(X1=np.array(x_dhd), random_state=3, y_true=y_true, num=2))
    debiases.append(debiased_frequency)

# use debias with the lowest precision for further study
db = debiases[np.argmin(precisions)]

## Confirm via Race-Specific Analogy Review

In [7]:
# Evaluate Stereotypical Analogy Generation
biasedAnalogies, biasedAnalogyGroups = generateAnalogies(roles, convert_legacy_to_keyvec(word_vectors))
hardDebiasedAnalogies, hardDebiasedAnalogyGroups = generateAnalogies(roles, convert_legacy_to_keyvec(new_vocab))
doubleHardDebiasedAnalogies, doubleHardDebiasedAnalogyGroups = generateAnalogies(roles, convert_legacy_to_keyvec(db))

In [8]:
limit = 20
print("Biased Analogies (0-" + str(limit) + ")")
for score, analogy, _ in biasedAnalogies[:limit]:
    print(score, analogy)

print("="*20, "\n\n")
print("Hard Debiased Analogies (0-" + str(limit) + ")")
for score, analogy, _ in hardDebiasedAnalogies[:limit]:
    print(score, analogy)

print("="*20, "\n\n")
print("Double-Hard Debiased Analogies (0-" + str(limit) + ")")
for score, analogy, _ in doubleHardDebiasedAnalogies[:limit]:
    print(score, analogy)

Biased Analogies (0-20)
0.7440957 asian is to laborer as caucasian is to maxim
0.7242655 asian is to laborer as caucasian is to quorum
0.722782 asian is to engineer as caucasian is to sociologist
0.72159934 black is to runner as caucasian is to fricative
0.7141854 asian is to laborer as caucasian is to forgery
0.71230006 black is to runner as caucasian is to rifleman
0.710995 black is to runner as caucasian is to alveolar
0.7099743 black is to runner as caucasian is to para
0.70986015 black is to runner as caucasian is to infinitive
0.70933586 asian is to engineer as caucasian is to astronomer
0.7057687 asian is to engineer as caucasian is to statistician
0.70569646 asian is to laborer as caucasian is to moniker
0.7055758 caucasian is to executive as asian is to australian
0.70499337 asian is to laborer as caucasian is to predicate
0.7049449 asian is to laborer as caucasian is to usury
0.7044244 asian is to laborer as caucasian is to claimant
0.70441103 black is to criminal as caucasia

In [9]:
print("Performing Evaluation")
print("="*20,)

evalTargets, evalAttrs = load_eval_terms('data/vocab/race_attributes_optm.json','role')

biasedMAC, biasedDistribution = multiclass_evaluation(word_vectors, evalTargets, evalAttrs)
print("Biased MAC:", biasedMAC)
debiasedMAC, debiasedDistribution = multiclass_evaluation(new_vocab, evalTargets, evalAttrs)
print("HARD MAC:", debiasedMAC)
doubledebiasedMAC, doubledebiasedDistribution = multiclass_evaluation(db, evalTargets, evalAttrs)
print("DOUBLE HARD MAC:", doubledebiasedMAC)

Performing Evaluation
Biased MAC: 0.8920888687350088
HARD MAC: 0.924706514661958
DOUBLE HARD MAC: 0.9222976140019051


# 2) Begin Utility Evaluation

## Concept Categorization

Cluster a set of words into different categorical subsets. Metric is fraction of the total number of the words that are correctly classified

### Concept

In [10]:
bench_names = ["EN-AP", "EN-ESSLI-2c", "EN-ESSLI-2b", "EN-ESSLI-1a", "EN-BATTIG", "EN-BLESS", ]
bench_paths = [os.path.join('data', 'concept_clustering', name) for name in bench_names]
benchmarks = create_bunches(bench_paths)

# show benchmark example to confirm
benchmarks['EN-AP']

{'X': array([['acne'],
        ['anthrax'],
        ['arthritis'],
        ['asthma'],
        ['cancer'],
        ['cholera'],
        ['cirrhosis'],
        ['diabetes'],
        ['eczema'],
        ['flu'],
        ['glaucoma'],
        ['hepatitis'],
        ['leukemia'],
        ['malnutrition'],
        ['meningitis'],
        ['plague'],
        ['rheumatism'],
        ['smallpox'],
        ['aluminium'],
        ['bismuth'],
        ['cadmium'],
        ['calcium'],
        ['carbon'],
        ['charcoal'],
        ['copper'],
        ['germanium'],
        ['helium'],
        ['hydrogen'],
        ['iron'],
        ['lithium'],
        ['magnesium'],
        ['neon'],
        ['nitrogen'],
        ['oxygen'],
        ['platinum'],
        ['potassium'],
        ['silver'],
        ['titanium'],
        ['zinc'],
        ['compulsion'],
        ['conscience'],
        ['deterrence'],
        ['disincentive'],
        ['dynamic'],
        ['ethics'],
        ['impulse'],
       

In [11]:
evaluate_cate(word_vectors, benchmarks)
print()
evaluate_cate(new_vocab, benchmarks)
print()
evaluate_cate(db, benchmarks)

Sample data from EN-AP, num of samples: 402 : "['acne']" is assigned class illness
exist 328 in 402
Cluster purity on EN-AP 0.5091463414634146
Sample data from EN-ESSLI-2c, num of samples: 45 : "['acquire']" is assigned class exchange-exchange
exist 45 in 45
Cluster purity on EN-ESSLI-2c 0.5555555555555556
Sample data from EN-ESSLI-2b, num of samples: 40 : "['chicken']" is assigned class HI
exist 40 in 40
Cluster purity on EN-ESSLI-2b 0.8500000000000001
Sample data from EN-ESSLI-1a, num of samples: 44 : "['bottle']" is assigned class tool-artifact
exist 44 in 44
Cluster purity on EN-ESSLI-1a 0.6590909090909091
Sample data from EN-BATTIG, num of samples: 5287 : "word" is assigned class toy
exist 3442 in 5287
Cluster purity on EN-BATTIG 0.3375944218477629
Sample data from EN-BLESS, num of samples: 200 : "['carp']" is assigned class water_animal
exist 193 in 200
Cluster purity on EN-BLESS 0.5181347150259068

Sample data from EN-AP, num of samples: 402 : "['acne']" is assigned class illnes

Control:

Embeddings given to us by the Black is to Criminal Paper.

In [12]:
biased_path = os.path.join('data', 'data_vocab_race_pre_trained.w2v')
biased_embeddings, embedding_dim = load_legacy_w2v(biased_path, 50)
# take out words that do not only contain letters
biased_word_vectors = pruneWordVecs(biased_embeddings)
evaluate_cate(biased_word_vectors, benchmarks)

print()

debiased_path = os.path.join('data','data_vocab_race_hard_debias.w2v')
debiased_embeddings, embedding_dim = load_legacy_w2v(debiased_path, 50)
# take out words that do not only contain letters
debiased_word_vectors = pruneWordVecs(debiased_embeddings)
evaluate_cate(debiased_word_vectors, benchmarks)

Sample data from EN-AP, num of samples: 402 : "['acne']" is assigned class illness
exist 328 in 402
Cluster purity on EN-AP 0.5091463414634146
Sample data from EN-ESSLI-2c, num of samples: 45 : "['acquire']" is assigned class exchange-exchange
exist 45 in 45
Cluster purity on EN-ESSLI-2c 0.5555555555555556
Sample data from EN-ESSLI-2b, num of samples: 40 : "['chicken']" is assigned class HI
exist 40 in 40
Cluster purity on EN-ESSLI-2b 0.8500000000000001
Sample data from EN-ESSLI-1a, num of samples: 44 : "['bottle']" is assigned class tool-artifact
exist 44 in 44
Cluster purity on EN-ESSLI-1a 0.7045454545454546
Sample data from EN-BATTIG, num of samples: 5287 : "word" is assigned class toy
exist 3442 in 5287
Cluster purity on EN-BATTIG 0.34166182452062754
Sample data from EN-BLESS, num of samples: 200 : "['carp']" is assigned class water_animal
exist 193 in 200
Cluster purity on EN-BLESS 0.538860103626943

Sample data from EN-AP, num of samples: 402 : "['acne']" is assigned class illnes

Conclusion:

Utility is preserved
*   AP - goes up
*   ESSLI-2c - stays the same
*   ESSLI-2b - stays the same
*   ESSLI-1a - goes up then back down
*   BATTIG - goes up (sometimes goes back down)
*   BLESS - goes up (sometimes goes back down)






## Analogy Evaluation

In [13]:
# Create text file for each category.
with open(os.path.join('data', 'analogies', 'EN-GOOGLE', 'EN-GOOGLE.txt'), "r") as file:
        lines = file.read().splitlines()

        questions = []
        answers = []
        category = []
        dummy = os.path.join('data', 'analogies', 'dummy.txt')

        f = open(dummy, "w")
        f.write("/n")
        for line in lines:
            if line.startswith(":"):
                
                # delete last "/n" and close file
                f.seek(f.tell()-1)
                f.truncate()
                f.close()

                # start new file for new category
                cat = line.lower().split()[1]
                f = open(os.path.join('data', 'analogies', f'{cat}.txt'), "w")

            else:
                f.write(line+'\n')

        os.remove(dummy)

In [14]:
print('-'*10, 'Biased', '-'*10)
evaluate_analogies(word_vectors)
print()
print('-'*10, 'Hard Biased', '-'*10)
evaluate_analogies(new_vocab)
print()
print('-'*10, 'Double Hard Biased', '-'*10)
evaluate_analogies(db)

---------- Biased ----------
family.txt:
ACCURACY TOP1: 70.26% (267/380)
gram1-adjective-to-adverb.txt:
ACCURACY TOP1: 8.06% (75/930)
gram2-opposite.txt:
ACCURACY TOP1: 17.86% (135/756)
gram3-comparative.txt:
ACCURACY TOP1: 68.62% (914/1332)
gram4-superlative.txt:
ACCURACY TOP1: 40.91% (432/1056)
gram5-present-participle.txt:
ACCURACY TOP1: 64.68% (683/1056)
gram7-past-tense.txt:
ACCURACY TOP1: 48.85% (762/1560)
gram8-plural.txt:
ACCURACY TOP1: 33.65% (424/1260)
gram9-plural-verbs.txt:
ACCURACY TOP1: 56.26% (319/567)
Questions seen/total: 46.24% (8897/19241)
Semantic accuracy: 70.26%  (267/380)
Syntactic accuracy: 43.96%  (3744/8517)
Total accuracy: 45.08%  (4011/8897)

---------- Hard Biased ----------
family.txt:
ACCURACY TOP1: 70.26% (267/380)
gram1-adjective-to-adverb.txt:
ACCURACY TOP1: 8.06% (75/930)
gram2-opposite.txt:
ACCURACY TOP1: 17.86% (135/756)
gram3-comparative.txt:
ACCURACY TOP1: 68.62% (914/1332)
gram4-superlative.txt:
ACCURACY TOP1: 40.91% (432/1056)
gram5-present-part

Control:

Embeddings given to us by the Black is to Criminal Paper.

In [15]:
evaluate_analogies(biased_word_vectors)
evaluate_analogies(debiased_word_vectors)

family.txt:
ACCURACY TOP1: 70.26% (267/380)
gram1-adjective-to-adverb.txt:
ACCURACY TOP1: 8.06% (75/930)
gram2-opposite.txt:
ACCURACY TOP1: 17.86% (135/756)
gram3-comparative.txt:
ACCURACY TOP1: 68.62% (914/1332)
gram4-superlative.txt:
ACCURACY TOP1: 40.91% (432/1056)
gram5-present-participle.txt:
ACCURACY TOP1: 64.68% (683/1056)
gram7-past-tense.txt:
ACCURACY TOP1: 48.85% (762/1560)
gram8-plural.txt:
ACCURACY TOP1: 33.65% (424/1260)
gram9-plural-verbs.txt:
ACCURACY TOP1: 56.26% (319/567)
Questions seen/total: 46.24% (8897/19241)
Semantic accuracy: 70.26%  (267/380)
Syntactic accuracy: 43.96%  (3744/8517)
Total accuracy: 45.08%  (4011/8897)
family.txt:
ACCURACY TOP1: 72.37% (275/380)
gram1-adjective-to-adverb.txt:
ACCURACY TOP1: 8.92% (83/930)
gram2-opposite.txt:
ACCURACY TOP1: 19.31% (146/756)
gram3-comparative.txt:
ACCURACY TOP1: 68.17% (908/1332)
gram4-superlative.txt:
ACCURACY TOP1: 40.91% (432/1056)
gram5-present-participle.txt:
ACCURACY TOP1: 63.07% (666/1056)
gram7-past-tense.tx